In [6]:

import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE(sitename,date)
    );
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [7]:
import requests

url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    for items in data['records']:
        sitenames.add(items['sitename'])

    sitenames = list(sitenames)
print(sitenames)

['萬里', '竹東', '恆春', '平鎮', '安南', '林口', '觀音', '中山', '宜蘭（頭城）', '橋頭', '陽明', '朴子', '小港', '屏東(枋山)', '崙背', '大城', '前鎮', '馬公', '頭份', '高雄（湖內）', '埔里', '美濃', '萬華', '新莊', '大園', '善化', '臺南', '土城', '富貴角', '新店', '基隆', '林園', '新營', '淡水', '楠梓', '線西', '臺東', '麥寮', '宜蘭', '鳳山', '西屯', '潮州', '三義', '桃園', '大同', '湖口', '古亭', '松山', '馬祖', '中壢', '斗六', '三重', '竹山', '花蓮', '大寮', '仁武', '臺南（麻豆）', '永和', '苗栗', '汐止', '臺西', '南投', '新港', '豐原', '新北(樹林)', '新竹', '左營', '彰化', '士林', '菜寮', '大里', '忠明', '冬山', '彰化（員林）', '關山', '嘉義', '二林', '沙鹿', '復興', '屏東', '前金', '屏東（琉球）', '板橋', '金門', '龍潭']


In [8]:
import datasource
datasource.get_selected_data(sitename='屏東(枋山)')

[['2024-11-04 12:00', '屏東縣', '39', '17', '良好', '22.260899', '120.651472'],
 ['2024-11-04 11:00', '屏東縣', '35', '8', '良好', '22.260899', '120.651472'],
 ['2024-11-04 11:00', '屏東縣', '35', '8', '良好', '22.260899', '120.651472'],
 ['2024-11-04 10:00', '屏東縣', '34', '18', '良好', '22.260899', '120.651472'],
 ['2024-11-04 10:00', '屏東縣', '34', '18', '良好', '22.260899', '120.651472'],
 ['2024-11-04 09:00', '屏東縣', '32', '5', '良好', '22.260899', '120.651472'],
 ['2024-11-04 09:00', '屏東縣', '32', '5', '良好', '22.260899', '120.651472'],
 ['2024-11-04 08:00', '屏東縣', '33', '6', '良好', '22.260899', '120.651472'],
 ['2024-11-04 08:00', '屏東縣', '33', '6', '良好', '22.260899', '120.651472'],
 ['2024-11-04 07:00', '屏東縣', '33', '5', '良好', '22.260899', '120.651472'],
 ['2024-11-04 07:00', '屏東縣', '33', '5', '良好', '22.260899', '120.651472'],
 ['2024-11-04 06:00', '屏東縣', '35', '4', '良好', '22.260899', '120.651472']]

In [9]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
    values ('屏東(枋山)','屏東縣', 34, '良好',18,'2024-11-04 10:00', 22.260899, 120.651472);
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [10]:
import sqlite3
conn = sqlite3.connect("AQI.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    with conn:
        cursor = conn.cursor()
        for items in data['records']:
            sitename = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] != '' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
            date = items['datacreationdate']
            lon = float(items['longitude']) if items['longitude'] != '' else 0.0
            lat = float(items['latitude']) if items['latitude'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                    values (?,?, ?, ?,?,?,?,?);
            '''
            cursor.execute(sql,(sitename, county, aqi, status,pm25,date,lat,lon))

        